# Project Overview

# Importing Basic Libraries

In [1]:
#These are the libraries I typically use in my analysis so I find it easier to import them all at once
#If I need more libraries I will import them as needed

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
plt.style.use('fivethirtyeight')
%matplotlib inline

C:\Users\kalstedh\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\kalstedh\AppData\Roaming\Python\Python37\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# First Look At the Dataset

In [2]:
#When we first imported the dataset it gave us a warning
#We will import it with low memory to avoid that warning

df = pd.read_csv('movies_metadata.csv', low_memory=False)

In [3]:
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# Dropping Irrelevant Columns

In [4]:
df.drop(columns = ['adult'], inplace = True)

In [5]:
df.drop(columns = ['imdb_id'], inplace = True)

In [6]:
df.drop(columns = ['original_title'], inplace = True)

In [7]:
df.drop(columns = ['video'], inplace = True)

In [8]:
df.drop(columns = ['homepage'], inplace = True)

# How to Handle Stringified JSON Columns Part One

In [9]:
#Here we are importing the JSON and AST modules
#AST is Abstract Syntax Trees. Helps Python applications to process trees of the Python abstract syntax grammar. 
#The abstract syntax itself might change with each Python release. 
#This module helps to find out programmatically what the current grammar looks like.

import json
import ast

In [10]:
json_col = ['belongs_to_collection', 'genres', 'production_countries', 'production_companies', 'spoken_languages']

In [11]:
#Double quotes for dictionary
#Single quotes for keys

json1 = "{'dog':3, 'cat': 5}"

In [12]:
#Apparently the double vs. single quotes did not work

json.loads(json1)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [13]:
#Single quotes for dictionary
#Double quotes for keys
#This actually works. 

json2 = '{"dog":3, "cat": 5}'

In [14]:
#Here we are replacing single quotes with double quotes
#We have converted a stringified JSON into a list

df.genres.apply(lambda x: json.loads(x.replace("'", '"')))[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [15]:
#So here ast evaluates a Python structure and see if it can be better suited as a different structure

ast.literal_eval(json1)

{'dog': 3, 'cat': 5}

In [16]:
df.genres.apply(ast.literal_eval)[0]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [17]:
df.genres = df.genres.apply(ast.literal_eval)

# How to Handle Stringified JSON Columns Part Two

In [18]:
#Here we are checking if each value is a string or not

df.belongs_to_collection.apply(lambda x: isinstance(x, str))

0         True
1        False
2         True
3        False
4         True
         ...  
45461    False
45462    False
45463    False
45464    False
45465    False
Name: belongs_to_collection, Length: 45466, dtype: bool

In [19]:
#Here we are applying a lambda function to the belongs to collection column
#This function will keep strings as strings and then convert non-strings to null values

df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [20]:
df.belongs_to_collection

0        {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                      NaN
2        {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                      NaN
4        {'id': 96871, 'name': 'Father of the Bride Col...
                               ...                        
45461                                                  NaN
45462                                                  NaN
45463                                                  NaN
45464                                                  NaN
45465                                                  NaN
Name: belongs_to_collection, Length: 45466, dtype: object

In [21]:
df.spoken_languages = df.spoken_languages.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [22]:
df.spoken_languages

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45461               [{'iso_639_1': 'fa', 'name': 'فارسی'}]
45462                    [{'iso_639_1': 'tl', 'name': ''}]
45463             [{'iso_639_1': 'en', 'name': 'English'}]
45464                                                   []
45465             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45466, dtype: object

In [23]:
df.production_countries = df.production_countries.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [24]:
df.production_countries

0        [{'iso_3166_1': 'US', 'name': 'United States o...
1        [{'iso_3166_1': 'US', 'name': 'United States o...
2        [{'iso_3166_1': 'US', 'name': 'United States o...
3        [{'iso_3166_1': 'US', 'name': 'United States o...
4        [{'iso_3166_1': 'US', 'name': 'United States o...
                               ...                        
45461               [{'iso_3166_1': 'IR', 'name': 'Iran'}]
45462        [{'iso_3166_1': 'PH', 'name': 'Philippines'}]
45463    [{'iso_3166_1': 'US', 'name': 'United States o...
45464             [{'iso_3166_1': 'RU', 'name': 'Russia'}]
45465     [{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]
Name: production_countries, Length: 45466, dtype: object

# How to Flatten Nested Columns

In [25]:
#Here we are applying a lambda function to the belongs to collection column
#We are looking at the name key in the dictionary format of the column
#For example: 'name' : 'Toy Story Collection' for the first movie in the database
#The function will keep the dictionary if that's how it's coded. If not, it will convert it to a null value

df.belongs_to_collection = df.belongs_to_collection.apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)

In [26]:
#Now here we are counting how many null values we have
#There are 40975, that's quite a lot

df.belongs_to_collection.value_counts(dropna = False).head()

NaN                              40975
The Bowery Boys                     29
Totò Collection                     27
James Bond Collection               26
Zatôichi: The Blind Swordsman       26
Name: belongs_to_collection, dtype: int64

In [27]:
#Here we are trying to join all of the genres that each film is categorized as
#Some have one, some have multiple
#Instead of having them separated by a comma, we will join them using | 

df.genres = df.genres.apply(lambda x: "|".join(i['name'] for i in x))

In [28]:
#As you can see, the genres for the first movie are now combined 

df.genres[0]

'Animation|Comedy|Family'

In [29]:
#Now here we are counting how many null values we have
#There are 2442 null values, basically an empty string

df.genres.value_counts(dropna = False).head()

Drama            5000
Comedy           3621
Documentary      2723
                 2442
Drama|Romance    1301
Name: genres, dtype: int64

In [30]:
#Here we are replacing the empty string with null values

df.genres.replace("", np.nan, inplace = True)

In [31]:
#Just like we did with genres, we will group the different languages spoken in movies togther

df.spoken_languages = df.spoken_languages.apply(lambda x: "|".join(i['name'] for i in x) if isinstance(x, list) else np.nan)

In [32]:
df.spoken_languages[0]

'English'

In [33]:
#Looks like there are 3952 empty strings

df.spoken_languages.value_counts(dropna = False).head()

English     22395
             3952
Français     1853
日本語          1289
Italiano     1218
Name: spoken_languages, dtype: int64

In [34]:
#Here we are replacing the empty string with null values

df.spoken_languages.replace("", np.nan, inplace = True)

In [35]:
#Just like we did with genres and spoken languages, we will group the different countries together

df.production_countries = df.production_countries.apply(lambda x: "|".join(i['name'] for i in x) 
                                                        if isinstance(x, list) else np.nan)

In [36]:
df.production_countries[0]

'United States of America'

In [37]:
#Looks like there are 6282 empty strings

df.production_countries.value_counts(dropna = False).head()

United States of America    17851
                             6282
United Kingdom               2238
France                       1654
Japan                        1356
Name: production_countries, dtype: int64

In [38]:
#Here we are replacing the empty string with null values

df.production_countries.replace("", np.nan, inplace = True)

# Cleaning Numerical Columns Part One

In [39]:
#Here we are trying to convert the budget data into a float format
#It will not let us because the data is too mixed

df.budget.astype('float')

ValueError: could not convert string to float: '/ff9qCepilowshEtG2GYWwzt2bs4.jpg'

In [40]:
#Here we are trying to convert the budget data into numerical form
#Once again like before we cannot do this due to too much data mixing

pd.to_numeric(df.budget)

ValueError: Unable to parse string "/ff9qCepilowshEtG2GYWwzt2bs4.jpg" at position 19730

In [41]:
#Now we are adding "coerce" 
#This allows us to convert most data into a numerical form and forces other data forms into a null value

df.budget = pd.to_numeric(df.budget, errors = 'coerce')

In [42]:
#Most frequest value is 0
#Seems pretty unrealistic. Movies tend to cost money to make

df.budget.value_counts(dropna = False)

0.0           36573
5000000.0       286
10000000.0      259
20000000.0      243
2000000.0       242
              ...  
9750000.0         1
7275000.0         1
78146652.0        1
280.0             1
1254040.0         1
Name: budget, Length: 1224, dtype: int64

In [43]:
#Here we are replacing 0s with NAN since the 0 might have been a placeholder for a unknown value

df.budget = df.budget.replace(0, np.nan)

In [44]:
#Here we are reducing the budget by a million so it helps with readability

df.budget = df.budget.div(1000000)

In [45]:
#The revenue column also seems to have a lot of 0s
#This makes more sense than the budget since a movie can indeed get 0 revenue, but seems unlikely that a majority have 0 revenue

df.revenue.value_counts(dropna = False)

0.0           38052
12000000.0       20
10000000.0       19
11000000.0       19
2000000.0        18
              ...  
36565280.0        1
439564.0          1
35610100.0        1
10217873.0        1
1413000.0         1
Name: revenue, Length: 6864, dtype: int64

In [46]:
#Here we are once again replacing the 0s with NAN

df.revenue = df.revenue.replace(0, np.nan)

In [47]:
#Here once again we are dividing the revenue column by one million to help with readability

df.revenue = df.revenue.div(1000000)

In [48]:
df.rename(columns = {'revenue': 'revenue_musd', 'budget': 'budget_musd'}, inplace = True)

# Cleaning Numerical Columns Part Two

In [49]:
#We have 1558 cases of a 0 minute runtime
#It is doubtful that many movies are 0 minutes long, if any
#We need to eliminate them from the dataset

df.runtime.value_counts(dropna = False).head()

90.0     2556
0.0      1558
100.0    1470
95.0     1412
93.0     1214
Name: runtime, dtype: int64

In [50]:
#Here we are replacing all 0 minute movies with NAN

df.runtime = df.runtime.replace(0, np.nan)

In [51]:
#Here we are trying to convert the movie id column to a numerical value
#Apparently we are unable due to mixed formatting in the column

pd.to_numeric(df.id)

ValueError: Unable to parse string "1997-08-20" at position 19730

In [52]:
#Here we are adding "coerce" to fight through the mixed formatting and force the id column to become numerical

df.id = pd.to_numeric(df.id, errors = 'coerce')

In [53]:
df.id.value_counts(dropna = False).head()

NaN         3
141971.0    3
11115.0     2
25541.0     2
15028.0     2
Name: id, dtype: int64

In [54]:
#It looks like we can't convert the popularity column to a numerical value just on its own. We have to coerce it

pd.to_numeric(df.popularity)

ValueError: Unable to parse string "Beware Of Frost Bites" at position 35587

In [55]:
#Here we are coercing the pd.to_numeric function

df.popularity = pd.to_numeric(df.popularity, errors = 'coerce')

In [56]:
df.popularity.value_counts(dropna= False).head()

0.000000    66
0.000001    56
0.000308    43
0.000220    40
0.000844    38
Name: popularity, dtype: int64

# Cleaning DateTime Columns

In [58]:
#Here we are simply viewing the release date column

df.release_date

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

In [59]:
#Now we are trying to convert the release date column into a datetime format
#It does not work

pd.to_datetime(df.release_date)

ValueError: Given date string not likely a datetime.

In [61]:
#Like we did with numerical data, we can coerce date data to fit our formatting scheme

df.release_date = pd.to_datetime(df.release_date, errors = 'coerce')

In [62]:
#So release date started as an object, but is now an integer

df.release_date.value_counts(dropna = False)

2008-01-01    136
2009-01-01    121
2007-01-01    118
2005-01-01    111
2006-01-01    101
             ... 
1957-09-26      1
1938-11-21      1
1936-08-19      1
2010-01-27      1
1917-10-21      1
Name: release_date, Length: 17334, dtype: int64

# Cleaing Text and String Columns

In [63]:
#There are many null values in the overview column
#There are also similar values. One is "no overview found" and the other is "no overview"

df.overview.value_counts(dropna = False).head()

NaN                             954
No overview found.              133
No Overview                       7
                                  5
No movie overview available.      3
Name: overview, dtype: int64

In [64]:
df.overview.replace("No overview found.", np.nan, inplace = True)

In [65]:
df.overview.replace("No Overview", np.nan, inplace = True)

In [66]:
df.overview.replace("No movie overview available", np.nan, inplace = True)

In [67]:
#Here we are replacing any empty strings with null values

df.overview.replace(" ", np.nan, inplace = True)

In [68]:
df.overview.replace("No overview yet.", np.nan, inplace = True)

In [69]:
#So now we have a lot of null values that we will have to adjust for later

df.overview.value_counts(dropna = False).head()

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               1101
A few funny little novels about different aspects of life.                                                                                                                                                                                                                                                                                                                                                                       

# Removing Duplicates

In [70]:
#Here we are identifying the duplicates in our dataset and sorting them by id
#As you can see there are several. 5511, 11115, etc

df[df.duplicated(keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
7345,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"[{'name': 'Fida cinematografica', 'id': 73}, {...",France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
9165,NaN,NaN,Crime|Drama|Thriller,5511.0,fr,Hitman Jef Costello is a perfectionist who alw...,9.091288,/cvNW8IXigbaMNo4gKEIps0NGnhA.jpg,"[{'name': 'Fida cinematografica', 'id': 73}, {...",France|Italy,1967-10-25,0.039481,105.0,Français,Released,There is no solitude greater than that of the ...,Le Samouraï,7.9,187.0
24844,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"[{'name': 'Andertainment Group', 'id': 2634}, ...",United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
14012,NaN,NaN,Comedy|Drama,11115.0,en,As an ex-gambler teaches a hot-shot college ki...,6.880365,/kHaBqrrozaG7rj6GJg3sUCiM29B.jpg,"[{'name': 'Andertainment Group', 'id': 2634}, ...",United States of America,2008-01-29,NaN,85.0,English,Released,NaN,Deal,5.2,22.0
22151,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,[],United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
14000,NaN,NaN,Action|Horror|Science Fiction,18440.0,en,When a comet strikes Earth and kicks up a clou...,1.436085,/tWCyKXHuSrQdLAvNeeVJBnhf1Yv.jpg,[],United States of America,2007-01-01,NaN,89.0,English,Released,NaN,Days of Darkness,5.0,5.0
8068,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,"[{'name': 'Filmfour', 'id': 6705}]",France|Germany|India|United Kingdom,2001-09-23,NaN,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
9327,NaN,NaN,Adventure|Animation|Drama|Action|Foreign,23305.0,en,"In feudal India, a warrior (Khan) who renounce...",1.967992,/9GlrmbZO7VGyqhaSR1utinRJz3L.jpg,"[{'name': 'Filmfour', 'id': 6705}]",France|Germany|India|United Kingdom,2001-09-23,NaN,86.0,हिन्दी,Released,NaN,The Warrior,6.3,15.0
17229,NaN,NaN,Drama,25541.0,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,[],Sweden|Denmark,2009-10-21,NaN,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0
23044,NaN,NaN,Drama,25541.0,da,Former Danish servicemen Lars and Jimmy are th...,2.587911,/q19Q5BRZpMXoNCA4OYodVozfjUh.jpg,[],Sweden|Denmark,2009-10-21,NaN,90.0,Dansk,Released,NaN,Brotherhood,7.1,21.0


In [71]:
#Here we are dropping the duplicates from our dataset

df.drop_duplicates(inplace = True)

In [72]:
df[df.duplicated(subset = 'id', keep = False)].sort_values(by = 'id')

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
33826,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",7.645827,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
5865,NaN,30.000000,Comedy|Crime|Drama|Romance|Thriller,4912.0,en,"Television made him famous, but his biggest hi...",11.331072,/o3Im9nPLAgtlw1j2LtpMebAotSe.jpg,"[{'name': 'Miramax Films', 'id': 14}, {'name':...",United States of America,2002-12-30,33.013805,113.0,English,Released,Some things are better left top secret.,Confessions of a Dangerous Mind,6.6,281.0
4114,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,10.264597,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...",Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,143.0
44821,Pokémon Collection,16.000000,Adventure|Fantasy|Animation|Action|Family,10991.0,ja,When Molly Hale's sadness of her father's disa...,6.480376,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,"[{'name': 'TV Tokyo', 'id': 3034}, {'name': '4...",Japan,2000-07-08,68.411275,93.0,English,Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,6.0,144.0
44826,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",6.080108,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,[],Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
5535,Pokémon Collection,NaN,Adventure|Fantasy|Animation|Science Fiction|Fa...,12600.0,ja,"All your favorite Pokémon characters are back,...",7.072301,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,[],Japan|United States of America,2001-07-06,28.023563,75.0,日本語,Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,5.7,82.0
15765,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.529879,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,"[{'name': 'Jafar Panahi Film Productions', 'id...",Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
11342,NaN,0.002500,Drama|Comedy|Foreign,13209.0,fa,"Since women are banned from soccer matches, Ir...",1.528960,/nfkOkpudNNIjRrf0mTFVoiGzHyc.jpg,"[{'name': 'Jafar Panahi Film Productions', 'id...",Iran,2006-05-26,NaN,93.0,فارسی,Released,NaN,Offside,6.7,27.0
10419,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.185256,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,"[{'name': 'Magnolia Pictures', 'id': 1030}, {'...",United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0
12066,NaN,1.600000,Drama|Crime|Mystery,14788.0,en,Set against the backdrop of a decaying Midwest...,3.008299,/w56oo9nREcF54sNXVYuE9QxZFjT.jpg,"[{'name': 'Magnolia Pictures', 'id': 1030}, {'...",United States of America,2005-09-03,NaN,73.0,English,Released,NaN,Bubble,6.4,36.0


In [73]:
df.drop_duplicates(subset = 'id', inplace = True)

In [74]:
#Here we are checking the id column value counts to verify if we have any duplicates
#Appears we were successful in eliminating all duplicates

df.id.value_counts(dropna = False)

862.0       1
74458.0     1
296206.0    1
107308.0    1
16247.0     1
           ..
44399.0     1
10138.0     1
32084.0     1
42191.0     1
461257.0    1
Name: id, Length: 45434, dtype: int64

# Handling Missing Values and Removing Observations

In [75]:
#Here we are counting all of the missing values in our dataset
#There are many in the belongs to collection and budget_musd columns
#Replacing missing values typically done for machine learning analysis

df.isna().sum()

belongs_to_collection    40946
budget_musd              36554
genres                    2442
id                           1
original_language           11
overview                  1101
popularity                   4
poster_path                386
production_companies         3
production_countries      6283
release_date                88
revenue_musd             38036
runtime                   1819
spoken_languages          3954
status                      85
tagline                  25033
title                        4
vote_average                 4
vote_count                   4
dtype: int64

In [76]:
#Here we are filtering just the movies with null values in the title column
#There are four

df[df.title.isna()]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19729,NaN,NaN,Action|Thriller|Drama,82663.0,en,British soldiers force a recently captured IRA...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19730,NaN,NaN,Carousel Productions|Vision View Entertainment...,NaN,104.0,Released,NaN,Midnight Man,False,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29502,Mardock Scramble Collection,NaN,Animation|Science Fiction,122662.0,ja,Third film of the Mardock Scramble series.,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35586,NaN,NaN,TV Movie|Action|Horror|Science Fiction,249260.0,en,A group of skiers are terrorized during spring...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
#Here we are dropping all rows that have missing id or title

df.dropna(subset = ['id', 'title'], inplace = True)

In [78]:
#Here we are converting the id column to an integer. it was originall a float

df.id = df.id.astype('int')

In [80]:
#Here we are filtering for variables with 6 missing values exactly

df[df.notna().sum(axis = 1) == 6]

,belongs_to_collection,budget_musd,genres,id,original_language,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue_musd,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


In [82]:
#Here we are dropping all variables with 10 or fewer missing variables

df.dropna(thresh = 10, inplace = True)

In [83]:
df.isna().sum()

belongs_to_collection    40911
budget_musd              36518
genres                    2412
id                           0
original_language           11
overview                  1081
popularity                   0
poster_path                358
production_companies         0
production_countries      6247
release_date                63
revenue_musd             38000
runtime                   1783
spoken_languages          3918
status                      75
tagline                  24997
title                        0
vote_average                 0
vote_count                   0
dtype: int64

# Final Cleaning Steps

In [85]:
#Here we are checking the value counts for status
#Looks like "Released" is the dominant status for most movies in this dataset

df.status.value_counts()

Released           44959
Rumored              229
Post Production       98
In Production         20
Planned               15
Canceled               2
Name: status, dtype: int64

In [86]:
#Becaused Released is the most common, we will simply make the dataset based on that status and eliminate the others

df = df.loc[df.status == 'Released'].copy()

In [87]:
#Here we are dropping the status column since that's been declared with Released

df.drop(columns = ['status'], inplace = True)

In [88]:
#Here we are creating column names for our new dataframe

col = ['id', 'title', 'tagline', 'release_date', 'genres', 'belongs_to_collection', 'original_language',
      'budget_musd', 'revenue_musd', 'production_companies', 'production_countries', 'vote_count', 'vote_average',
      'popularity', 'runtime', 'overview', 'spoken_languages', 'poster_path']

In [89]:
#Here we are adding the column names to the dataframe

df = df.loc[:, col]

In [90]:
#Here is what our dataframe now looks like

df.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,"[{'name': 'Pixar Animation Studios', 'id': 3}]",United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,[{'name': 'Twentieth Century Fox Film Corporat...,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,"[{'name': 'Sandollar Productions', 'id': 5842}...",United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,/e64sOI48hQXyru7naBFyssKFxVd.jpg


In [91]:
df.reset_index(drop = True, inplace = True)

In [92]:
df.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,production_countries,vote_count,vote_average,popularity,runtime,overview,spoken_languages,poster_path
0,862,Toy Story,NaN,1995-10-30,Animation|Comedy|Family,Toy Story Collection,en,30.0,373.554033,"[{'name': 'Pixar Animation Studios', 'id': 3}]",United States of America,5415.0,7.7,21.946943,81.0,"Led by Woody, Andy's toys live happily in his ...",English,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg
1,8844,Jumanji,Roll the dice and unleash the excitement!,1995-12-15,Adventure|Fantasy|Family,NaN,en,65.0,262.797249,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",United States of America,2413.0,6.9,17.015539,104.0,When siblings Judy and Peter discover an encha...,English|Français,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg
2,15602,Grumpier Old Men,Still Yelling. Still Fighting. Still Ready for...,1995-12-22,Romance|Comedy,Grumpy Old Men Collection,en,NaN,NaN,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",United States of America,92.0,6.5,11.712900,101.0,A family wedding reignites the ancient feud be...,English,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg
3,31357,Waiting to Exhale,Friends are the people who let you be yourself...,1995-12-22,Comedy|Drama|Romance,NaN,en,16.0,81.452156,[{'name': 'Twentieth Century Fox Film Corporat...,United States of America,34.0,6.1,3.859495,127.0,"Cheated on, mistreated and stepped on, the wom...",English,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg
4,11862,Father of the Bride Part II,Just When His World Is Back To Normal... He's ...,1995-02-10,Comedy,Father of the Bride Collection,en,NaN,76.578911,"[{'name': 'Sandollar Productions', 'id': 5842}...",United States of America,173.0,5.7,8.387519,106.0,Just when George Banks has recovered from his ...,English,/e64sOI48hQXyru7naBFyssKFxVd.jpg


In [93]:
#Here we are looking at the first value for poster path

df.poster_path[0]

'/rhIRbceoE9lR4veEXuwCC2wARtG.jpg'

In [94]:
base_poster_url = 'http://image.tmdb.org/t/p/w185/'

In [96]:
df.poster_path = "<img src='" + base_poster_url + df.poster_path + "' style ='height:100px:'>"

In [97]:
df.poster_path

0        <img src='http://image.tmdb.org/t/p/w185//rhIR...
1        <img src='http://image.tmdb.org/t/p/w185//vzmL...
2        <img src='http://image.tmdb.org/t/p/w185//6ksm...
3        <img src='http://image.tmdb.org/t/p/w185//16XO...
4        <img src='http://image.tmdb.org/t/p/w185//e64s...
                               ...                        
44954    <img src='http://image.tmdb.org/t/p/w185//jlds...
44955    <img src='http://image.tmdb.org/t/p/w185//xZkm...
44956    <img src='http://image.tmdb.org/t/p/w185//d5bX...
44957    <img src='http://image.tmdb.org/t/p/w185//aorB...
44958    <img src='http://image.tmdb.org/t/p/w185//s5Uk...
Name: poster_path, Length: 44959, dtype: object